In [1]:
# Import required packages/libraries:
import os
import sys
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql import DataFrame
from functools import reduce

# Dependencies for Regression Algorithms:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import PCA
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import IsotonicRegression

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
32,application_1606590675617_0033,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Define sql context:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Load train and test datasets form S3:

df = spark.read.parquet("s3://projectmlldsongs/songs_data_split_v2/type=full/")
train_df = spark.read.parquet("s3://projectmlldsongs/songs_data_split_v2/type=train/")
test_df = spark.read.parquet("s3://projectmlldsongs/songs_data_split_v2/type=test/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
##Isotonic Regression Model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Convert Rows of dataframe to Dense Vectors:
assembler = VectorAssembler(
    inputCols=['artist_familiarity','duration','loudness','key_confidence','key','end_of_fade_in',
            'time_signature_confidence','tempo','mode'],
    outputCol='features')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# List containing values of k:
pca_list = [1,2,3,4,5,6,7,8,9]

# Empty list for storing rmse values:
rmse_list = []

for num_features in pca_list:
    
    # Define pca object:
    pca = PCA(k=num_features, inputCol="features",outputCol="features_pca")
    
    # Create a pipeline to sequentially perform operations on the training set:
    pipeline_pca = Pipeline(stages=[assembler, pca])
    
    # Fit pca model on full dataset:
    pcamodel = pipeline_pca.fit(df)
    
    # Apply pca on training data and test data:
    train_predictions = pcamodel.transform(train_df)
    pca_train_data = train_predictions.select("features_pca","song_hotttnesss")
    test_predictions = pcamodel.transform(test_df)
    pca_test_data = test_predictions.select("features_pca","song_hotttnesss")
    
    # Define isotonic regression model:
    isoreg = IsotonicRegression(labelCol='song_hotttnesss', featuresCol='features_pca')
    
    #Train model on lower-dimensional features:
    isomodel = isoreg.fit(pca_train_data)
    
    # Evaluate on test data:
    predictions = isomodel.transform(pca_test_data)
    
    # Select (prediction, true label) and compute test error
    evaluator = RegressionEvaluator(
        labelCol="song_hotttnesss", predictionCol="prediction", metricName="rmse")
    
    rmse = evaluator.evaluate(predictions)
    
    # Store rmse value in a list:
    rmse_list.append(rmse)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
print(rmse_list)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.094594581209568, 0.09459458120956801, 0.094594581209568, 0.09459458120956801, 0.09459458120956801, 0.094594581209568, 0.09459458120956801, 0.09459458120956801, 0.094594581209568]